[Amirhossein Mahmoudi](https://ammahmoudi.github.io)
# 3-step Adams–Moulton Method ( functional iteration mode vs newton method mode)
solution to [Numerical Analysis (R. Burden, Aires, A. Burden) [10th Ed.]](https://www.amazon.com/Richard-Burden-Numerical-Analysis-Hardcover/dp/B00SB3UL20) Section 5.6 question 13

![5_6_13](images/5_6_13.png)

In [155]:
import numpy as np
import pandas as pd

In [156]:
f = lambda t, y: np.exp(y)
d_f=lambda t,y:np.exp(y)
f_actual = lambda t: 1-np.log(1-np.exp(1)*t)

#t0
t = 0
#y0
y = 1
#end point
p = 0.2
#step size
h = 0.01
epsilon=1E-10

In [157]:
def newton(f,Df,x0,epsilon,max_iter):
    '''Approximate solution of f(x)=0 by Newton's method.

    Parameters
    ----------
    f : function
        Function for which we are searching for a solution f(x)=0.
    Df : function
        Derivative of f(x).
    x0 : number
        Initial guess for a solution f(x)=0.
    epsilon : number
        Stopping criteria is abs(f(x)) < epsilon.
    max_iter : integer
        Maximum number of iterations of Newton's method.

    Returns
    -------
    xn : number
        Implement Newton's method: compute the linear approximation
        of f(x) at xn and find x intercept by the formula
            x = xn - f(xn)/Df(xn)
        Continue until abs(f(xn)) < epsilon and return xn.
        If Df(xn) == 0, return None. If the number of iterations
        exceeds max_iter, then return None.
    '''
    xn = x0
    for n in range(0,max_iter):
        fxn = f(xn)
        if abs(fxn) < epsilon:
            print('Found solution after',n,'iterations.')
            return xn,n
        Dfxn = Df(xn)
        if Dfxn == 0:
            print('Zero derivative. No solution found.')
            return None,n
        xn = xn - fxn/Dfxn
    print('Exceeded maximum iterations. No solution found.')
    return None,None

In [158]:
def adamsMoulton3(t, p, f_actual, h, f,mode):
    n = int((p - t)/h)
    time = [t+h*i for i in range(n+2)]
    g = lambda wFixPoint,w,f,i : w[i] + (h/24)*(9*f(time[i+1], wFixPoint)+ 19*f(time[i], w[i]) - 5*f(time[i-1], w[i - 1])+ f(time[i-2], w[i - 2]))
    Dg=lambda wFixedPoint,w,f,i:1+(h/24)*(9*f(time[i+1], wFixPoint)+ 19*f(time[i], w[i]) - 5*f(time[i-1], w[i - 1])+ f(time[i-2], w[i - 2]))
    #Initialize w
    w = np.zeros_like(time)
    #iterations counter
    iterations_counter=np.zeros_like(time)
    y_actual=[f_actual(time[i]) for  i in range(len(time))]
   
    if(mode=='functional_iteration'):
        w[0:3] = [f_actual(time[0]),f_actual(time[1]),f_actual(time[2])] #Set initial values w(1), w(2), w(3) to y(0), y(0.01), y(0.02)
        for i in range(2,n):
            #we need to perform t fixpoint iteration for w(i+1) to get w_i+1 = g(w_i+1)
            # start fixpoint iteration for w_i+1 with w_i as described
            wFixPoint = w[i]
            counter=0
            while (abs(wFixPoint-g(wFixPoint,w,f,i))> 1E-10):
                    wFixPoint = g(wFixPoint,w,f,i)
                    counter=counter+1
                    #print('\t doing a fix point iteration... w =',wFixPoint)
            iterations_counter[i+1]=counter
            w[i+1] = wFixPoint
            # print('time =', time[i+1])
            # print('w =', w[i+1])
    elif mode=='newton':
        for i in range(2,n):
            #we need to perform t fixpoint iteration for w(i+1) to get w_i+1 = g(w_i+1)
            # start fixpoint iteration for w_i+1 with w_i as described
            wFixPoint = w[i]
            f_t=lambda w_i:w[i] + (h/24)*(9*f(time[i+1], w_i)+ 19*f(time[i], w[i]) - 5*f(time[i-1], w[i - 1])+ f(time[i-2], w[i - 2]))
            Df_t=lambda w_i:w[i] + (h/24)*(9*f(time[i+1], w_i)+ 19*f(time[i],w[i]) - 5*f(time[i-1], w[i - 1])+ f(time[i-2], w[i - 2]))
            wFixPoint,counter=newton(f_t,Df_t,wFixPoint,10E-10,10)

            iterations_counter[i+1]=counter
            w[i+1] = wFixPoint
            # print('time =', time[i+1])
            # print('w =', w[i+1])
         
    return np.c_[time,w,y_actual,iterations_counter]
    


In [159]:
result=adamsMoulton3(t,p,f_actual,h,f,'functional_iteration')
pd.DataFrame(result,columns=['time','w','y','iterations']).head(21)

,time,w,y,iterations
0,0.00,1.000000,1.000000,0.0
1,0.01,1.027559,1.027559,0.0
2,0.02,1.055899,1.055899,0.0
3,0.03,1.085066,1.085066,5.0
4,0.04,1.115109,1.115109,5.0
5,0.05,1.146083,1.146083,5.0
6,0.06,1.178047,1.178047,5.0
7,0.07,1.211067,1.211066,5.0
8,0.08,1.245214,1.245213,5.0
9,0.09,1.280568,1.280568,5.0
